## Import of relevant python modules

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import plotly.offline
import plotly.graph_objects as go
from scipy.stats import ks_2samp
from sbemdb import SBEMDB
from cleandb import clean_db
from mapping import Mapping

np.warnings.filterwarnings('ignore')
db = SBEMDB()

#Note that clean_db returns synaptic partners of DE3-R which both have been identified with VSD and EM
#Comment out next line to visualize synapses from the larger set of EM identified partner neurons
db = clean_db(db)
x,y,z = db.segments(444)
mapping = Mapping()

## Extract coordinates and other information on pre- and postsynaptic sites

In [3]:
(xx, yy, zz, pretid, posttid, synid, prenid, postnid) = db.synapses('pre.tid!=444 and post.tid=444',extended = True)

In [4]:
synapses = {}
sids = {}
for i in range(len(xx)):
    if synid[i] in sids:
        continue;
    tid = pretid[i]
    if tid not in synapses: synapses[tid] = []
    synapses[tid].append([xx[i], yy[i], zz[i]])
    sids[synid[i]] = True;

## Plot connectome of DE3-R and synapses from presynaptic partner neurons

In [7]:
def plot_graph(synapses,min_synapses = 1):
    each_tree_scatter = []
    soma = []
    for tid in synapses:
        if len(synapses[tid]) < min_synapses: continue
        xs = []
        ys = []
        zs = []
        cs = []
        for coords in synapses[tid]:
            xs.append(coords[0])
            ys.append(coords[1])
            zs.append(coords[2])
    
        each_tree_scatter.append(go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers',
                                   hoverinfo='text',
                                   hovertext=f'{mapping.sbem2can[tid]}',           
                                   name=f'Canonical name: {mapping.sbem2can[tid]}',
                                   marker=dict(
                                        size=3.5,                # set color to an array/list of desired values
                                        opacity=0.8
                                    )))
    

    nodes = db.nodexyz(f'tid==444 and typ==1') # (x, y, z, nid)
    soma.append(go.Scatter3d(x=nodes[0], y=nodes[1], z=nodes[2], 
                                   name='Soma',
                                   hoverinfo=[],
                                   hovertext=[],
                                   mode='markers',
                                   marker=dict(
                                        color= "rgb(0,0,0)",
                                        size=6,                # set color to an array/list of desired values                                    
                                        opacity=0.8
                                    )))
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='lines',
                                   opacity=1,
                                   name='DE3',
                                   marker=dict(
                                       color=1,
                    
                                    ))] + soma + each_tree_scatter)
    
    
    fig.update_layout(title="DE3-R motor neuron with synapses of partner neurons",
                      showlegend=True,
        scene=dict(
            xaxis=dict(nticks=5, range=[50, 300], showbackground=False, showticklabels=False, title=''),
            yaxis=dict(nticks=13, range=[50, 750], showbackground=False, showticklabels=False, title=''),
            zaxis=dict(nticks=6, range=[50, 350], showbackground=False, showticklabels=False, title=''),
            aspectmode='data'
        ),
        scene_camera=dict(
            eye=dict(x=-0.8, y=-0.8, z=0.05),
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0)
        )
    )   

    
    plotly.offline.plot(fig, filename='Visualization_DE-3R.html', auto_open=True)
    fig.show()
    
plot_graph(synapses);
